In [1]:
import pandas as pd
import duckdb

In [2]:
# Connect to Database
con = duckdb.connect(database='database.db', read_only=False)

In [3]:
nets_pub_data = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/hq_companies_public_maris.dta')
nets_all_data = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/hq_all_maris.dta')
# ['HQDuns', 'HQCompany', 'HQAdress','HQZipCode', 'HQCity', 'HQState', 'HQArea', 'HQPhone', 'Latitude', 'Longitude', 'LevelCode']

In [4]:
nets_all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82365757 entries, 0 to 82365756
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   latitude   float32
 1   longitude  float32
 2   hqduns     int32  
 3   hqcompany  object 
dtypes: float32(2), int32(1), object(1)
memory usage: 1.5+ GB


In [5]:
compustat_all = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/compustat.dta')
# ['gvkey', 'conm', 'add1', 'addzip', 'city', 'loc', 'state', 'phone']
compustat_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42979 entries, 0 to 42978
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   gvkey      42979 non-null  int32  
 1   datadate   42979 non-null  object 
 2   fyear      42808 non-null  float64
 3   indfmt     42979 non-null  object 
 4   consol     42979 non-null  object 
 5   popsrc     42979 non-null  object 
 6   datafmt    42979 non-null  object 
 7   tic        42979 non-null  object 
 8   hqcompany  42979 non-null  object 
 9   curcd      42979 non-null  object 
 10  costat     42979 non-null  object 
 11  hqaddress  42979 non-null  object 
 12  add2       42979 non-null  object 
 13  add3       42979 non-null  object 
 14  add4       42979 non-null  object 
 15  hqzipcode  42979 non-null  object 
 16  city       42979 non-null  object 
 17  conml      42979 non-null  object 
 18  state      42979 non-null  object 
 19  hqzip4     33973 non-null  float64
dtypes: flo

In [6]:
compustat_all.sample(100)

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,hqcompany,curcd,costat,hqaddress,add2,add3,add4,hqzipcode,city,conml,state,hqzip4
14594,17145,1988-09-30,1988.0,INDL,C,D,STD,WAFD,WAFD INC,USD,A,425 Pike Street,,,,98101,Seattle,WaFd Inc,WA,9810.0
30668,65846,2005-12-31,2005.0,INDL,C,D,STD,HIT.UN,FIERA HIGH INCOME TRUST,CAD,I,"1981 McGill College Avenue, Suite 1500",,,,H3A 0H5,Montreal,Fiera High Income Trust,QC,NaN
5169,6254,1967-12-31,1967.0,INDL,C,D,STD,JAIR,JET AIR FREIGHT,USD,I,,,,,,,Jet Air Freight,,NaN
28084,61164,1993-12-31,1993.0,INDL,C,D,STD,VLNT,VIDEOLAN TECHNOLOGIES INC,USD,I,"11403 Bluegrass Parkway, Suite 400",,,,40299,Louisville,Videolan Technologies Inc,KY,4029.0
20426,28011,2016-12-31,2016.0,INDL,C,D,STD,MEU.,MACKENZIE MAX DVRS DEV EURO,CAD,A,180 Queen Street West,,,,M5V 3K1,Toronto,Mackenzie Maximum Diversification Developed Eu...,ON,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32274,112592,2001-02-28,2000.0,INDL,C,D,STD,FRMO,FRMO CORP,USD,I,"1 North Lexington Avenue, Suite 12C",,,,10601,White Plains,FRMO Corp,NY,1060.0
22073,30219,1994-01-31,1993.0,INDL,C,D,STD,BTH,BLYTH INC,USD,I,One East Weaver Street,,,,06831,Greenwich,Blyth Inc,CT,683.0
29546,63873,1991-12-31,1991.0,INDL,C,D,STD,SMCI.1,SIMULATION SCIENCES INC,USD,I,"601 Valencia Avenue, Suite 100",,,,92823-6346,Brea,Simulation Sciences Inc,CA,9282.0
14467,16977,1988-12-31,1988.0,INDL,C,D,STD,HPBC,HOME PORT BANCORP INC,USD,I,104 Pleasant Street,,,,02554,Nantucket,Home Port Bancorp Inc,MA,255.0


In [7]:
compustat_geocoded = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/compustat_geocoded_final.dta')
# ['gvkey', 'hq_address_compustat', 'count', 'lat', 'lon']
print(compustat_geocoded.info())
compustat_geocoded

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42980 entries, 0 to 42979
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey                 42980 non-null  int32  
 1   hq_address_compustat  42980 non-null  object 
 2   count                 42980 non-null  float32
 3   lat                   41999 non-null  float64
 4   lon                   41999 non-null  float64
dtypes: float32(1), float64(2), int32(1), object(1)
memory usage: 1.3+ MB
None


,gvkey,hq_address_compustat,count,lat,lon
0,1000,,1.0,NaN,NaN
1,1001,1924 South Utica Avenue Tulsa OK,2.0,-95.968066,36.134328
2,1002,124 Industry Lane Hunt Valley MD,3.0,-76.641662,39.467797
3,1003,7700 Hall Street St. Louis MO,4.0,-90.225770,38.707338
4,1004,1100 North Wood Dale Road Wood Dale IL,5.0,-87.981054,41.986749
...,...,...,...,...,...
42975,351491,Via Puglia n. 35 Turin,42976.0,7.723420,45.106830
42976,351590,Fasanenweg 10 Leinfelden-Echterdingen,42977.0,9.145069,48.707416
42977,352262,Hamilton,42978.0,175.281099,-37.789474
42978,353444,The Heights Building 5 Weybridge,42979.0,-0.462543,51.352960


In [8]:
con.execute('DROP TABLE IF EXISTS compustat_geocoded')
con.execute('CREATE TABLE compustat_geocoded AS SELECT * FROM compustat_geocoded')
con.sql('SELECT DISTINCT COUNT(gvkey) FROM compustat_geocoded;')

┌──────────────┐
│ count(gvkey) │
│    int64     │
├──────────────┤
│        42980 │
└──────────────┘

-- START OF COMPUSTAT PREPROCESSING --

In [9]:
con.execute('DROP TABLE IF EXISTS compustat')
con.execute('CREATE TABLE compustat AS SELECT DISTINCT gvkey, hqcompany FROM compustat_all')
con.sql('SELECT DISTINCT COUNT(*) FROM compustat;')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        42979 │
└──────────────┘

In [10]:
"""
Extract entries of compustat_geocoded that are not in all_data_cleaned
"""
query_str = """
SELECT compustat_geocoded.gvkey
FROM compustat_geocoded
LEFT JOIN compustat ON compustat_geocoded.gvkey = compustat.gvkey
WHERE compustat.gvkey IS NULL;
"""
con.sql(query_str).fetchdf()
# (duckdb.sql(query_str).to_df()).to_excel('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/preprocessed_data/missing_entries_compustat.xlsx', index=False)


,gvkey
0,9007


In [11]:
print(compustat_all.loc[compustat_all['gvkey'] == 1001])
compustat_geocoded.loc[compustat_geocoded['gvkey'] == 1001]

   gvkey    datadate   fyear indfmt consol popsrc datafmt    tic  \
1   1001  1978-12-31  1978.0   INDL      C      D     STD  AMFD.   

                 hqcompany curcd costat                hqaddress add2 add3  \
1  A & M FOOD SERVICES INC   USD      I  1924 South Utica Avenue             

  add4 hqzipcode   city                    conml state  hqzip4  
1          94104  Tulsa  A & M Food Services Inc    OK  9410.0  


,gvkey,hq_address_compustat,count,lat,lon
1,1001,1924 South Utica Avenue Tulsa OK,2.0,-95.968066,36.134328


In [12]:
"""
In this cell, we want to join the geo date from compustat_geocoded with the other data from compustat_all
"""

con.execute('ALTER TABLE compustat ADD COLUMN lat DECIMAL(9,6)')
con.execute('ALTER TABLE compustat ADD COLUMN lon DECIMAL(9,6)')

# TODO: Note that lat and lon are switched in compustat_geocoded. This is probably a mistake in the geocoding process
sql_query = """
UPDATE compustat
SET lat = compustat_geocoded.lon,
    lon = compustat_geocoded.lat
FROM compustat_geocoded
WHERE compustat.gvkey = compustat_geocoded.gvkey;
"""
con.execute(sql_query)

In [13]:
"""
Appearantly, there are 2196 ETFS in the compustat dataset. We want to get rid of them.
"""
sql_query = """
SELECT *
FROM compustat
WHERE REGEXP_MATCHES(hqcompany, '\\bETF\\b');
"""
con.sql(sql_query)

┌────────┬──────────────────────────────┬──────────────┬──────────────┐
│ gvkey  │          hqcompany           │     lat      │     lon      │
│ int32  │           varchar            │ decimal(9,6) │ decimal(9,6) │
├────────┼──────────────────────────────┼──────────────┼──────────────┤
│  12932 │ INVESCO DORSEY WRI SM MO ETF │    41.822742 │   -88.031668 │
│  18112 │ VANGUARD EM MKTS GOVT BD ETF │    40.051521 │   -75.504835 │
│  18987 │ CI MRNGSTR US VLUE INDEX ETF │    43.653524 │   -79.383907 │
│  19011 │ FIDELITY MSCI UTILITIES ETF  │    37.789152 │  -122.395878 │
│  19279 │ SPDR MSCI EM BEYOND BRIC ETF │    42.344753 │   -71.052116 │
│  19884 │ INVESCO INT BUYBACK ACHI ETF │    41.822742 │   -88.031668 │
│  21119 │ SPROTT GOLD MINERS ETF       │    39.736700 │  -104.987340 │
│  22186 │ BMO INTL DIVIDEND ETF        │    43.671294 │   -79.384772 │
│  22905 │ HORIZONS CDN INSIDER IDX ETF │    43.646957 │   -79.384009 │
│  23115 │ PACER PCFIC AFR HGH INCM ETF │    33.618181 │  -117.8

In [14]:
"""
We delete all entries that contain the substring 'ETF'
"""
sql_query = """
DELETE FROM compustat
WHERE REGEXP_MATCHES(hqcompany, '\\bETF\\b');
"""
con.execute(sql_query)
con.sql('SELECT * FROM compustat;').fetchdf()

,gvkey,hqcompany,lat,lon
0,1000,A & E PLASTIK PAK INC,NaN,NaN
1,1002,AAI CORP,39.467797,-76.641662
2,1019,AFA PROTECTIVE SYSTEMS INC,40.804278,-73.517759
3,1026,AID INC-DEL CL A,NaN,NaN
4,1038,AMC ENTERTAINMENT INC -OLD,39.103020,-94.583564
...,...,...,...,...
39777,326688,NVENT ELECTRIC PLC,51.507408,-0.127699
39778,327832,ARGO BLOCKCHAIN PLC,51.516670,-0.138510
39779,334298,PROPERTYGURU GROUP LIMITED,1.317400,103.892270
39780,345980,CONTEXTLOGIC INC,37.790486,-122.400962


In [15]:
"""
In this Cell we want to preprocess the names of the compustat datasets
"""
# most common substrings in company names
compustat_substring = ['INC', 'CORP', 'LTD', 'GROUP', 'CO', 'CORP']
#TODO: Maybe get rid of things like 'ENTERPRISES', 'PROPERTIES', 'INTERNATIONAL', 'TECHNOLOGIES'
#TODO: Mauybe ged rid of terms like -old or -llp
#TODO replace terms like technologie through special characters like $ or ˆ, since they are often used in company names
# other: ['etf', 'financial', 'bancorp', 'holdings', 'energy']

# query_str = """
# UPDATE compustat
#     SET conm = REGEXP_REPLACE(conm, '(\\bLLC\\b)|(\\bINC\\b)|(\\bLTD\\b)|(\\bGROUP\\b)|(\\bCO\\b)|(\\bCORP\\b)|(\\s+)|(\\-)', '', 'g')
# """
#TOOD: We replace longer terms with their abbreviations, to reduce their weight in the name, but keeping them, to improve matching score
query_str = """
UPDATE compustat
SET hqcompany = TRIM(
    REGEXP_REPLACE(
        REGEXP_REPLACE(
            REGEXP_REPLACE(
                REGEXP_REPLACE(
                    REGEXP_REPLACE(
                        REGEXP_REPLACE(
                            REGEXP_REPLACE(
                                REGEXP_REPLACE(
                                    REGEXP_REPLACE(
                                        REGEXP_REPLACE(
                                            REGEXP_REPLACE(
                                                REGEXP_REPLACE( 
                                                    REGEXP_REPLACE(
                                                        REGEXP_REPLACE(
                                                            REGEXP_REPLACE(
                                                                REGEXP_REPLACE(
                                                                    REGEXP_REPLACE(
                                                                        REGEXP_REPLACE(
                                                                            REGEXP_REPLACE(hqcompany, 'INCORPORATED', 'INC'),
                                                                        'LIMITED', '`LTD'),
                                                                    'CORPORATION', 'CORP'),
                                                                'COMPANY', 'CO'),
                                                            'HOLDINGS', 'HLDGS'),
                                                        'GROUP', 'GRP'),
                                                    'INTERNATIONAL', 'INTL'),
                                                'TECHNOLOGIES', '@'),
                                            'THERAPEUTICS', '^'),
                                        'PHARMACEUTICALS', '$'),
                                    'INDUSTRIES', 'IND'),
                                'TECHNOLOGY', '!'),
                            'FINANCIAL', ')'),
                        'COMMUNICATIONS', '('),
                    'PROPERTIES', '\'),
                '\\b-\\b', ''),
            '\\s+', '', 'g'),
        'BANCORPINC', '}'),
    'HLDGSINC', '{')
    )
"""
con.execute(query_str)

In [16]:
# """
# Clean up the phone numbers from whitespaces and dashes
# """
# query_str = """
# UPDATE compustat
#     SET phone = REGEXP_REPLACE(phone, '(\\s+)|-', '', 'g')
# """
# con.execute(query_str)
# con.sql('SELECT * FROM compustat WHERE hqcompany LIKE \'%!%\' LIMIT 10').fetchdf()


-- END OF COMPUSTAT PREPROCESSING --


-- START OF NETS PREPROCESSING --


In [17]:
con.execute('DROP TABLE IF EXISTS nets_all')
con.execute('CREATE TABLE nets_all AS SELECT hqduns, hqcompany, latitude, longitude, FROM nets_all_data')
#con.sql('SELECT * FROM nets;').fetchdf().info()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [18]:
"""
In this Cell we want to preprocess the names of the NETS datasets

NETS_substring = ['LLC', 'INC', 'OF', 'GROUP', 'SERVICES', 'CO', 'CORP']
other: ['a', 'construction', 'enterprises']
"""
#TODO: get rid of things like 'ENTERPRISES', 'PROPERTIES', 'INTERNATIONAL', 'TECHNOLOGIES', 'INCORPORATED', 'CORPORATION', 'COMPANY since acronyms like inc. arent that common in nets.
#TODO: Maybe we need more preprocessing for NETS since there are a lot of entries that are not in compustat

# query_str = """
# UPDATE nets
#     SET hqcompany = REGEXP_REPLACE(hqcompany, '(\\bLLC\\b)|(\\bINC\\b)|(\\bLTD\\b)|(\\bGROUP\\b)|(\\bCO\\b)|(\\bCORP\\b)|(\\s+)|(\\-)', '', 'g')
# """
query_str = """
UPDATE nets_all
SET hqcompany = TRIM(
    REGEXP_REPLACE(
        REGEXP_REPLACE(
            REGEXP_REPLACE(
                REGEXP_REPLACE(
                    REGEXP_REPLACE(
                        REGEXP_REPLACE(
                            REGEXP_REPLACE(
                                REGEXP_REPLACE(
                                    REGEXP_REPLACE(
                                        REGEXP_REPLACE(
                                            REGEXP_REPLACE(
                                                REGEXP_REPLACE( 
                                                    REGEXP_REPLACE(
                                                        REGEXP_REPLACE(
                                                            REGEXP_REPLACE(
                                                                REGEXP_REPLACE(
                                                                    REGEXP_REPLACE(
                                                                        REGEXP_REPLACE(
                                                                            REGEXP_REPLACE(hqcompany, 'INCORPORATED', 'INC'),
                                                                        'LIMITED', '`LTD'),
                                                                    'CORPORATION', 'CORP'),
                                                                'COMPANY', 'CO'),
                                                            'HOLDINGS', 'HLDGS'),
                                                        'GROUP', 'GRP'),
                                                    'INTERNATIONAL', 'INTL'),
                                                'TECHNOLOGIES', '@'),
                                            'THERAPEUTICS', '^'),
                                        'PHARMACEUTICALS', '$'),
                                    'INDUSTRIES', 'IND'),
                                'TECHNOLOGY', '!'),
                            'FINANCIAL', ')'),
                        'COMMUNICATIONS', '('),
                    'PROPERTIES', '\'),
                '\\b-\\b', ''),
            '\\s+', '', 'g'),
        'BANCORPINC', '}'),
    'HLDGSINC', '{')
    )
"""
con.execute(query_str)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [19]:
# """
# In this Cell we want to extract the area code and the phone number from the NETS data
# and concatenate them to one column which then will be added to the NETS data.
# Afterwards we will compare the phone numbers of the two datasets and see if they match.
# """
# con.execute('ALTER TABLE nets ADD COLUMN hqphone_concatenated VARCHAR(255)')
# query_str = """
# UPDATE nets
# SET hqphone_concatenated = REGEXP_REPLACE(
#             REGEXP_REPLACE((CONCAT(hqarea, hqphone)), '(\\s+)|-', '', 'g')
#           , '^$', '-1');
# """
# con.execute(query_str)

In [20]:
"""
In this Cell we will fix the longitudes of the NETS data
"""
#TODO: The coordinates are not correct in our NETS sample.  Google maps gives us -119.xxxx
#TODO: Lat-long coordinates for cities in United States are in range: Latitude from 19.50139 to 64.85694 and longitude from -161.75583 to -68.01197. (Source: https://latitudelongitude.org/us/)
# TODO maybe we need to adjust the coordinates according to levelcode
sql_query = """
    UPDATE nets_all
    SET longitude = -longitude;
"""
con.execute(sql_query)

-- NETS PUB PREPROCESSING --

In [21]:
con.execute('DROP TABLE IF EXISTS nets_pub')
con.execute('CREATE TABLE nets_pub AS SELECT hqduns, hqcompany, latitude, longitude, FROM nets_pub_data')

In [22]:
"""
In this Cell we want to preprocess the names of the NETS datasets
"""
query_str = """
UPDATE nets_pub
SET hqcompany = TRIM(
    REGEXP_REPLACE(
        REGEXP_REPLACE(
            REGEXP_REPLACE(
                REGEXP_REPLACE(
                    REGEXP_REPLACE(
                        REGEXP_REPLACE(
                            REGEXP_REPLACE(
                                REGEXP_REPLACE(
                                    REGEXP_REPLACE(
                                        REGEXP_REPLACE(
                                            REGEXP_REPLACE(
                                                REGEXP_REPLACE( 
                                                    REGEXP_REPLACE(
                                                        REGEXP_REPLACE(
                                                            REGEXP_REPLACE(
                                                                REGEXP_REPLACE(
                                                                    REGEXP_REPLACE(
                                                                        REGEXP_REPLACE(
                                                                            REGEXP_REPLACE(hqcompany, 'INCORPORATED', 'INC'),
                                                                        'LIMITED', '`LTD'),
                                                                    'CORPORATION', 'CORP'),
                                                                'COMPANY', 'CO'),
                                                            'HOLDINGS', 'HLDGS'),
                                                        'GROUP', 'GRP'),
                                                    'INTERNATIONAL', 'INTL'),
                                                'TECHNOLOGIES', '@'),
                                            'THERAPEUTICS', '^'),
                                        'PHARMACEUTICALS', '$'),
                                    'INDUSTRIES', 'IND'),
                                'TECHNOLOGY', '!'),
                            'FINANCIAL', ')'),
                        'COMMUNICATIONS', '('),
                    'PROPERTIES', '\'),
                '\\b-\\b', ''),
            '\\s+', '', 'g'),
        'BANCORPINC', '}'),
    'HLDGSINC', '{')
    )
"""
con.execute(query_str)

In [23]:
"""
In this Cell we will fix the longitudes of the NETS data
"""
#TODO: The coordinates are not correct in our NETS sample.  Google maps gives us -119.xxxx
#TODO: Lat-long coordinates for cities in United States are in range: Latitude from 19.50139 to 64.85694 and longitude from -161.75583 to -68.01197. (Source: https://latitudelongitude.org/us/)
# TODO maybe we need to adjust the coordinates according to levelcode
sql_query = """
    UPDATE nets_pub
    SET longitude = -longitude;
"""
con.execute(sql_query)

In [24]:
# con.execute('SELECT * FROM nets').fetch_df().to_excel('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/preprocessed_data/nets_preprocessed.xlsx', index=False)
# con.execute('SELECT * FROM compustat').fetchdf().to_excel('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/preprocessed_data/compustat_preprocessed.xlsx', index=False)

In [25]:
con.commit()
con.close()